In [ ]:
def newtons_method_large(A, y, lambda_reg, tol=1e-6, max_iter=100):
    N, d = A.shape
    x = np.zeros((d, 1))  # Starting point
    for k in range(max_iter):
        # Gradient
        grad = A.T @ (A @ x - y) + lambda_reg * x
        # Hessian
        H = A.T @ A + lambda_reg * np.eye(d)
        # Check if Hessian is invertible
        if np.linalg.cond(H) > 1 / np.finfo(float).eps:
            raise ValueError("Hessian is poorly conditioned or singular")
        # Newton step
        delta_x = -np.linalg.solve(H, grad)
        x += delta_x
        # Convergence check
        if np.linalg.norm(delta_x) < tol:
            break
    return x


In [ ]:
from scipy.optimize import minimize

def objective_f_large(x, A, y, lambda_reg):
    x = x.reshape(-1, 1)
    return 0.5 * np.linalg.norm(A @ x - y)**2 + 0.5 * lambda_reg * np.linalg.norm(x)**2

def bfgs_method_large(A, y, lambda_reg):
    d = A.shape[1]
    result = minimize(
        objective_f_large,
        np.zeros(d),
        args=(A, y, lambda_reg),
        method='BFGS',
        options={'disp': False}
    )
    return result.x.reshape(-1, 1)


In [ ]:
import numpy as np
import timeit

# Initialization
np.random.seed(10)
N = 200
ds = [1000, 5000, 10000, 20000, 25000, 50000, 100000, 200000, 500000, 1000000]
lambda_reg = 0.001
eps = np.random.randn(N, 1)  # Random noise

# Results storage
results = []

for d in ds:
    print(f"Running experiments for dimension: {d}")
    A = np.random.randn(N, d)
    # Normalize the columns
    A = A / np.linalg.norm(A, axis=0)
    xorig = np.ones((d, 1))
    y = A @ xorig + eps

    # Newton Method
    try:
        start_newton = timeit.default_timer()
        x_opt_newton = newtons_method_large(A, y, lambda_reg)
        newton_time = timeit.default_timer() - start_newton
        newton_fit_error = np.linalg.norm(A @ x_opt_newton - y)**2
        newton_diff = np.linalg.norm(x_opt_newton - xorig)**2
    except Exception as e:
        newton_time, newton_fit_error, newton_diff = "Failed", "Failed", "Failed"

    # BFGS Method
    try:
        start_bfgs = timeit.default_timer()
        x_opt_bfgs = bfgs_method_large(A, y, lambda_reg)
        bfgs_time = timeit.default_timer() - start_bfgs
        bfgs_fit_error = np.linalg.norm(A @ x_opt_bfgs - y)**2
        bfgs_diff = np.linalg.norm(x_opt_bfgs - xorig)**2
    except Exception as e:
        bfgs_time, bfgs_fit_error, bfgs_diff = "Failed", "Failed", "Failed"

    # Append results
    results.append({
        'dimension': d,
        'newton_time': newton_time,
        'newton_fit_error': newton_fit_error,
        'newton_diff': newton_diff,
        'bfgs_time': bfgs_time,
        'bfgs_fit_error': bfgs_fit_error,
        'bfgs_diff': bfgs_diff
    })

    # Stop if either method takes too long or fails
    if newton_time == "Failed" and bfgs_time == "Failed":
        print("Both methods failed for this dimension. Stopping further experiments.")
        break

# Tabulate results
import pandas as pd
results_df = pd.DataFrame(results)
print(results_df)


Running experiments for dimension: 1000
Running experiments for dimension: 5000


KeyboardInterrupt: 